# OSN postprocessing

Spracovanie výstupu algoritmu do formy využiteľnej pre ďalšie analýzy

In [ ]:
import pandas as pd

In [ ]:
# Aby fungovalo nacitanie spolocnych funkcii, je nutne ich nainstalovat cez `pip install .` na urovni, kde sa nachadza pyproject.toml

from OSN_common.nacitanie_dat import nacitaj_vsetku_starostlivost, nacitaj_vystup_algoritmu, nacitaj_vsetku_starostlivost_s_ms, nacitaj_zoznam_ms, nacitaj_programovy_profil
from OSN_common.pomocne_funkcie import prirad_vekovu_kategoriu, zisti_uroven_ms


In [ ]:
_rok = 2023
_verzia = 'v2024.2'

# Pripojenie výstupu algoritmu ku všetkej starostlivosti

In [ ]:
vystup = nacitaj_vystup_algoritmu(_rok, _verzia)
final = nacitaj_vsetku_starostlivost(_rok)

In [ ]:
final = final.merge(vystup, on='id_hp', how='left')

## Pripojenie úrovní medicínskych služieb

In [ ]:
final['vekova_kategoria'] = final['vek_roky'].apply(prirad_vekovu_kategoriu)
final['ms_list'] = final['ms'].apply(lambda s: s.split('~') if pd.notna(s) else s)


In [ ]:
zoznam_ms = nacitaj_zoznam_ms(_verzia)
zoznam_ms = zoznam_ms[~zoznam_ms['zdielana_ms']]

In [ ]:
final_exploded = final[['id_hp', 'pzs_8', 'vekova_kategoria', 'ms_list']].explode('ms_list')
final_exploded = final_exploded.merge(zoznam_ms, left_on='ms_list', right_on='kod_ms', how='left')
final_exploded['uroven_ms'] = final_exploded.apply(zisti_uroven_ms, axis=1).astype('Int16')

In [ ]:
urovne_ms = final_exploded.groupby('id_hp')['uroven_ms'].apply(lambda z: '~'.join([str(x) for x in z]))

In [ ]:
final = final.merge(urovne_ms, on='id_hp')
final['hlavna_ms'] = final['ms_list'].str[0]

## MS na úrovne programu 

In [ ]:
programovy_profil = nacitaj_programovy_profil(2024, _verzia)

In [ ]:
final['cislo_programu'] = final['hlavna_ms'].str.split('-').str[0].str[1:].astype(int)
final['uroven_hlavnej_ms'] = final['uroven_ms'].str.split('~').str[0]

In [ ]:
pivot = pd.pivot_table(final, values='id_hp', index=['cislo_programu'], columns='uroven_hlavnej_ms', aggfunc='count').fillna(0).astype(int)
pivot = pivot.merge(programovy_profil[['cislo_programu', 'nazov_programu']], how='left', on='cislo_programu').drop_duplicates()[['cislo_programu', 'nazov_programu', '1', '2', '3', '4', '5']]

# Export

In [ ]:
final.drop(columns=['vekova_kategoria', 'ms_list', 'hlavna_ms']).to_csv(f'Vystupy/osn_vsetka_starostlivost_{_rok}_ms_{_verzia}.csv', sep=';', index=False)
final.groupby(['hlavna_ms', 'pzs_8'])['id_hp'].count().rename('pocet_hospitalizacnych_pripadov').to_csv(f'Vystupy/pocet_MS_na_nemocnicu_iba_hlavne_{_rok}_{_verzia}.csv')
final_exploded.rename({'ms_list': 'ms'}, axis=1).groupby(['ms', 'pzs_8'])['id_hp'].count().rename('pocet_hospitalizacnych_pripadov').to_csv(f'Vystupy/pocet_MS_na_nemocnicu_{_rok}_{_verzia}.csv')


In [ ]:
pivot.to_csv(f'Vystupy/pocet_MS_na_uroven_programu_iba_hlavne_{_rok}_{_verzia}.csv', index=False, encoding='utf_8_sig')

## Kontrola s pôvodným výstupom

In [ ]:
final_old = nacitaj_vsetku_starostlivost_s_ms(_rok, _verzia)

merged = final.merge(final_old[['id_hp', 'ms']], on='id_hp', how='outer')

In [ ]:
len(merged[merged['ms_x'] != merged['ms_y']])